In [1]:
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import geopandas

**Trafik Verisi - Eksik Veri Tamamlama**

In [2]:
TrafikDataDF=pd.read_csv('İstanbulTraficData.csv',sep=';')
print("Veri Sayısı : "+str(TrafikDataDF["TrafficIndex"].count()))

Veri Sayısı : 43481


In [3]:
TrafikDataDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43481 entries, 0 to 43480
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TrafficIndex      43481 non-null  int64  
 1   TrafficIndexDate  43481 non-null  object 
 2   Date              43481 non-null  object 
 3   Time              43481 non-null  object 
 4   Day               43481 non-null  int64  
 5   Month             43481 non-null  int64  
 6   Year              43481 non-null  int64  
 7   Week              43481 non-null  int64  
 8   daysofweek        43236 non-null  float64
 9   day_name          43236 non-null  object 
 10  is_weekend        43236 non-null  object 
 11  Days_Of_Week      43481 non-null  int64  
 12  Day_Name          43481 non-null  object 
 13  Is_Weekend_Flag   43481 non-null  bool   
dtypes: bool(1), float64(1), int64(6), object(6)
memory usage: 4.4+ MB


In [4]:
#Eksik Gün Hesaplama
TrafikDataDF['TrafficIndexDate']=TrafikDataDF['TrafficIndexDate'].apply(pd.to_datetime)
Days=pd.to_datetime("today").date()-TrafikDataDF.TrafficIndexDate.dt.date.max()
Days=Days.days
print("Eksik Gün Sayısı : "+str(Days))

Eksik Gün Sayısı : 0


In [5]:
#Eksik Saat Hesaplama
if (pd.to_datetime("today").time()==TrafikDataDF.loc[TrafikDataDF['TrafficIndexDate'].dt.date==TrafikDataDF.TrafficIndexDate.dt.date.max()].TrafficIndexDate.dt.time.max().strftime("%H:%M"))==False:
    Days=Days+1
    print("Değer Güncellendi. Yeni Değer : "+str(Days))

Değer Güncellendi. Yeni Değer : 1


In [6]:
#IBB veri setinden son 910 günün verisini 30 Dakika aralıklı olarak çekiyoruz
if Days!=0:
    response = requests.get("https://api.ibb.gov.tr/tkmservices/api/TrafficData/v1/TrafficIndexHistory/"+str(Days)+"/30M")
    TrafikDataDF=pd.concat([TrafikDataDF,pd.DataFrame(response.json())])
    TrafikDataDF=TrafikDataDF.drop_duplicates(subset=['TrafficIndexDate'])
    print("Veri Güncellendi. Yeni Veri Sayısı : "+str(TrafikDataDF["TrafficIndex"].count()))

Veri Güncellendi. Yeni Veri Sayısı : 43530


In [ ]:
TrafikDataDF['TrafficIndexDate']=TrafikDataDF['TrafficIndexDate'].apply(pd.to_datetime)

TrafikDataDF['Date'] = TrafikDataDF.TrafficIndexDate.dt.date
TrafikDataDF['Time'] = TrafikDataDF.TrafficIndexDate.dt.time
TrafikDataDF["Day"]=TrafikDataDF.TrafficIndexDate.dt.day
TrafikDataDF["Month"]=TrafikDataDF.TrafficIndexDate.dt.month
TrafikDataDF["Year"]=TrafikDataDF.TrafficIndexDate.dt.year
TrafikDataDF["Week"]=TrafikDataDF.TrafficIndexDate.dt.week
TrafikDataDF['Days_Of_Week'] = TrafikDataDF['TrafficIndexDate'].dt.dayofweek
TrafikDataDF["Day_Name"]=TrafikDataDF.TrafficIndexDate.dt.day_name()
TrafikDataDF['Is_Weekend_Flag'] = np.where(TrafikDataDF['Day_Name'].isin(['Sunday', 'Saturday']), "True","False")

In [16]:
TrafikDataDF.count()

TrafficIndex        43530
TrafficIndexDate    43530
Date                43530
Time                43530
Day                 43530
Month               43530
Year                43530
Week                43530
daysofweek          43236
day_name            43236
is_weekend          43236
Days_Of_Week        43530
Day_Name            43530
Is_Weekend_Flag     43530
dtype: int64

In [20]:
TrafikDataDF.to_csv('İstanbulTraficData.csv',sep=';',index=False)

**Araç Takip**

In [9]:
docs=['https://data.ibb.gov.tr/dataset/ed3160cd-1756-40c7-9063-c3306ba71b06/resource/15e1332d-e476-4899-b3b9-8574ac8f6eb8/download/gunluk-arac-saym.csv','https://data.ibb.gov.tr/dataset/ed3160cd-1756-40c7-9063-c3306ba71b06/resource/b08038fd-b087-489f-910f-7c0aaf848325/download/gunluk-arac-saym.csv']

In [10]:
for doc in docs:
    if doc==docs[0]:
        AracDataDF=pd.read_csv(doc,sep=';',encoding='iso-8859-1',dtype={"Arac Sayisi":str})
    else:
        AracDataDF=pd.concat([AracDataDF,pd.read_csv(doc,sep=';',encoding='iso-8859-1',dtype={"Arac Sayisi":str})])

In [11]:
AracDataDF["Kordinat"]= AracDataDF["X Koordinati"].str.cat(AracDataDF["Y Koordinati"], sep =",")

In [12]:
AracDataDF=AracDataDF.dropna(subset=['Arac Sayisi'])

In [ ]:
AracDataDF['Tarih']=AracDataDF['Tarih'].apply(pd.to_datetime)

AracDataDF["Day"]=AracDataDF.Tarih.dt.day
AracDataDF["Month"]=AracDataDF.Tarih.dt.month
AracDataDF["Year"]=AracDataDF.Tarih.dt.year
AracDataDF["Week"]=AracDataDF.Tarih.dt.week
AracDataDF['Days_Of_Week'] = AracDataDF['Tarih'].dt.dayofweek
AracDataDF["Day_Name"]=AracDataDF.Tarih.dt.day_name()
AracDataDF['Is_Weekend_Flag'] = np.where(AracDataDF['Day_Name'].isin(['Sunday', 'Saturday']), "True","False")

In [14]:
AracDataDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 711424 entries, 0 to 96475
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   Tarih            711424 non-null  datetime64[ns]
 1   Sensor Adi       711403 non-null  object        
 2   X Koordinati     705677 non-null  object        
 3   Y Koordinati     705677 non-null  object        
 4   Arac Sayisi      711424 non-null  object        
 5   Unnamed: 5       0 non-null       float64       
 6   Kordinat         705677 non-null  object        
 7   Day              711424 non-null  int64         
 8   Month            711424 non-null  int64         
 9   Year             711424 non-null  int64         
 10  Week             711424 non-null  int64         
 11  Days_Of_Week     711424 non-null  int64         
 12  Day_Name         711424 non-null  object        
 13  Is_Weekend_Flag  711424 non-null  object        
dtypes: datetime64[ns](1),

In [15]:
AracDataDF.to_csv('İstanbulAracData.csv',sep=';',index=False)